In [3]:
import sys

sys.path.append('..')
import os
import torch
import pandas as pd
import numpy as np
from deep_logic.models.relu_nn import XReluNN
from deep_logic.models.psi_nn import PsiNetwork
from deep_logic.models.tree import XDecisionTreeClassifier
from deep_logic.utils.base import set_seed
from deep_logic.utils.metrics import F1Score
from deep_logic.models.general_nn import XGeneralNN
from deep_logic.utils.datasets import ConceptToTaskDataset
from deep_logic.utils.data import get_splits_train_val_test
from deep_logic.logic.base import test_multi_class_explanation

results_dir = 'results/cub'
if not os.path.isdir(results_dir):
    os.makedirs(results_dir)

## Define loss, metrics and saved metrics

In [2]:
loss = torch.nn.CrossEntropyLoss()
metric = F1Score()

methods = []
splits = []
explanations = []
explanations_inv = []
model_accuracies = []
explanation_accuracies = []
explanation_accuracies_inv = []
elapsed_times = []
elapsed_times_inv = []

## Loading CUB data

In [3]:
dataset = ConceptToTaskDataset("../data/CUB_200_2011/")
concept_names = dataset.attribute_names
print("Concept names", concept_names)
n_features = dataset.n_attributes
print("Number of features", n_features)

Concept names ['has_bill_shape_dagger' 'has_bill_shape_hooked_seabird'
 'has_bill_shape_allpurpose' 'has_bill_shape_cone' 'has_wing_color_brown'
 'has_wing_color_grey' 'has_wing_color_yellow' 'has_wing_color_black'
 'has_wing_color_white' 'has_wing_color_buff' 'has_upperparts_color_brown'
 'has_upperparts_color_grey' 'has_upperparts_color_yellow'
 'has_upperparts_color_black' 'has_upperparts_color_white'
 'has_upperparts_color_buff' 'has_underparts_color_brown'
 'has_underparts_color_grey' 'has_underparts_color_yellow'
 'has_underparts_color_black' 'has_underparts_color_white'
 'has_underparts_color_buff' 'has_breast_pattern_solid'
 'has_breast_pattern_striped' 'has_breast_pattern_multicolored'
 'has_back_color_brown' 'has_back_color_grey' 'has_back_color_yellow'
 'has_back_color_black' 'has_back_color_white' 'has_back_color_buff'
 'has_tail_shape_notched_tail' 'has_upper_tail_color_brown'
 'has_upper_tail_color_grey' 'has_upper_tail_color_black'
 'has_upper_tail_color_white' 'has_uppe

## Training Hyperparameters

In [4]:
epochs = 200
l_r = 0.001
lr_scheduler = True
top_k_explanations = 1
simplify = True
seeds = [*range(10)]
print("Seeds", seeds)

Seeds [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


## Decision Tree

In [5]:
for seed in seeds:
    print("Seed", seed)
    set_seed(seed)

    train_data, val_data, test_data = get_splits_train_val_test(dataset)
    print(train_data.indices)

    device = torch.device("cpu")

    print("Training Tree Classifier...")
    model = XDecisionTreeClassifier(n_classes=dataset.n_classes, n_features=n_features)

    results = model.fit(train_data, val_data, metric=metric, save=False)
    print(results)

    accuracy = model.evaluate(test_data)
    print("Test model accuracy", accuracy)

    formulas, times = [], []
    for i, class_to_explain in enumerate(dataset.classes):
        formula, elapsed_time = model.get_global_explanation(i, concept_names,
                                                             return_time=True)
        formulas.append(formula), times.append(elapsed_time)
        print(f"{class_to_explain} <-> {formula}")
        print("Elapsed time", elapsed_time)

    methods.append("Tree")
    splits.append(seed)
    explanations.append(formulas[0])
    explanations_inv.append(formulas[1])
    model_accuracies.append(accuracy)
    explanation_accuracies.append(accuracy)
    explanation_accuracies_inv.append(accuracy)
    elapsed_times.append(np.mean(times))
    elapsed_times_inv.append(np.mean(times))

results_tree = pd.DataFrame({
    'method': methods,
    'split': splits,
    'explanation': explanations,
    'explanation_inv': explanations_inv,
    'model_accuracy': model_accuracies,
    'explanation_accuracy': explanation_accuracies,
    'explanation_accuracy_inv': explanation_accuracies_inv,
    'elapsed_time': elapsed_times,
    'elapsed_time_inv': elapsed_times_inv,
})
results_tree.to_csv(os.path.join(results_dir, 'results_tree.csv'))
print(results_tree)

Seed 0
[    0     2     3 ... 11784 11785 11787]
Training Tree Classifier...


KeyboardInterrupt: 

## Relu NN

In [ ]:
for seed in seeds:
    print("Seed", seed)
    set_seed(seed)

    train_data, val_data, test_data = get_splits_train_val_test(dataset)
    print(train_data.indices)

    x_val = torch.tensor(dataset.attributes[val_data.indices])
    y_val = torch.tensor(dataset.targets[val_data.indices])
    x_test = torch.tensor(dataset.attributes[test_data.indices])
    y_test = torch.tensor(dataset.targets[test_data.indices])

    # Network structures
    l1_weight = 1e-5
    hidden_neurons = [200, 100]

    # Setting device
    device = torch.device("cpu") if torch.cuda.is_available() else torch.device("cpu")
    set_seed(seed)
    print(f"Training Relu NN...")
    model = XReluNN(n_classes=dataset.n_classes, n_features=n_features,
                    hidden_neurons=hidden_neurons, loss=loss, l1_weight=l1_weight)

    results = model.fit(train_data, val_data, epochs=epochs, l_r=l_r, verbose=False,
                        metric=metric, lr_scheduler=lr_scheduler, device=device, save=False)
    print(results)
    accuracy = model.evaluate(test_data)
    print("Test Model accuracy", accuracy)

    formulas, times, exp_accuracies = [], [], []
    for i, class_to_explain in enumerate(dataset.classes):
        formula, elapsed_time = model.get_global_explanation(x_val, y_val, i,
                                                             topk_explanations=top_k_explanations,
                                                             concept_names=concept_names,
                                                             simplify=True, return_time=True)
        exp_accuracy, _ = test_multi_class_explanation(formula, i, x_test, y_test,
                                                       metric=metric, concept_names=concept_names)
        formulas.append(formula), times.append(elapsed_time), exp_accuracies.append(exp_accuracy)
        print(f"{class_to_explain} <-> {formula}")
        print("Elapsed time", elapsed_time)
        print("Explanation accuracy", exp_accuracy)

    methods.append("Relu")
    splits.append(seed)
    explanations.append(formulas[0])
    explanations_inv.append(formulas[1])
    model_accuracies.append(accuracy)
    explanation_accuracies.append(np.mean(exp_accuracies))
    explanation_accuracies_inv.append(np.mean(exp_accuracies))
    elapsed_times.append(np.mean(times))
    elapsed_times_inv.append(np.mean(times))

results = pd.DataFrame({
    'method': methods,
    'split': splits,
    'explanation': explanations,
    'explanation_inv': explanations_inv,
    'model_accuracy': model_accuracies,
    'explanation_accuracy': explanation_accuracies,
    'explanation_accuracy_inv': explanation_accuracies_inv,
    'elapsed_time': elapsed_times,
    'elapsed_time_inv': elapsed_times_inv,
})
results_relu = results[results['method'] == "Relu"]
results_relu.to_csv(os.path.join(results_dir, 'results_relu.csv'))
print(results_relu)

Seed 0
[    1     2     3 ... 11785 11786 11787]
Training Relu NN...
     Tot losses  Train accs   Val accs  Best epoch
0      3.453620   40.223238  71.483287         111
1      0.872653   85.835161  86.960132         111
2      0.616138   89.948673  90.472145         111
3      0.539422   91.155048  90.353495         111
4      0.501061   91.783590  91.491650         111
..          ...         ...        ...         ...
195    0.152954   98.230533  91.025225         111
196    0.151461   98.317097  91.069614         111
197    0.150820   98.326918  91.025624         111
198    0.148075   98.430718  91.186047         111
199    0.149086   98.379939  90.448129         111

[200 rows x 4 columns]
Test Model accuracy 92.39222316145394
001.Black_footed_Albatross <-> has_bill_shape_hooked_seabird & has_breast_pattern_solid & has_eye_color_black & has_bill_length_about_the_same_as_head & has_size_medium_9__16_in & has_back_pattern_solid & has_wing_pattern_solid & ~has_bill_shape_allpurpose 

## PSI NN

In [ ]:
for seed in seeds:
    print("Seed", seed)
    set_seed(seed)

    train_data, val_data, test_data = get_splits_train_val_test(dataset)
    print(train_data.indices)

    x_test = torch.tensor(dataset.attributes[test_data.indices])
    y_test = torch.tensor(dataset.targets[test_data.indices])

    # Network structures
    l1_weight = 5e-6
    hidden_neurons = [10, 5]
    fan_in = 3
    lr_psi = 0.01

    # Setting device
    device = torch.device("cpu") if torch.cuda.is_available() else torch.device("cpu")
    set_seed(seed)

    print("Training Psi NN...")
    model = PsiNetwork(dataset.n_classes, n_features, hidden_neurons, loss,
                       l1_weight, fan_in=fan_in)

    results = model.fit(train_data, val_data, epochs=epochs, l_r=lr_psi, verbose=False,
                        metric=metric, lr_scheduler=lr_scheduler, device=device, save=False)
    print(results)
    accuracy = model.evaluate(test_data, metric=metric)
    print("Test model accuracy", accuracy)

    formulas, times, exp_accuracies = [], [], []
    for i, class_to_explain in enumerate(dataset.classes):
        formula, elapsed_time = model.get_global_explanation(i, concept_names,
                                                             simplify=True, return_time=True)
        exp_accuracy, _ = test_multi_class_explanation(formula, i, x_test, y_test,
                                                       metric=metric, concept_names=concept_names)
        formulas.append(formula), times.append(elapsed_time), exp_accuracies.append(exp_accuracy)
        print(f"{class_to_explain} <-> {formula}")
        print("Elapsed time", elapsed_time)
        print("Explanation accuracy", exp_accuracy)

    methods.append("Psi")
    splits.append(seed)
    explanations.append(formulas[0])
    explanations_inv.append(formulas[1])
    model_accuracies.append(accuracy)
    explanation_accuracies.append(np.mean(exp_accuracies))
    explanation_accuracies_inv.append(np.mean(exp_accuracies))
    elapsed_times.append(np.mean(times))
    elapsed_times_inv.append(np.mean(times))

results = pd.DataFrame({
    'method': methods,
    'split': splits,
    'explanation': explanations,
    'explanation_inv': explanations_inv,
    'model_accuracy': model_accuracies,
    'explanation_accuracy': explanation_accuracies,
    'explanation_accuracy_inv': explanation_accuracies_inv,
    'elapsed_time': elapsed_times,
    'elapsed_time_inv': elapsed_times_inv,
})
results_psi = results[results['method'] == "Psi"]
results_psi.to_csv(os.path.join(results_dir, 'results_psi.csv'))
print(results_psi)

## Mu NN

In [ ]:
for seed in seeds:
    print("Seed", seed)
    set_seed(seed)

    train_data, val_data, test_data = get_splits_train_val_test(dataset)
    print(train_data.indices)

    x_val = torch.tensor(dataset.attributes[val_data.indices])
    y_val = torch.tensor(dataset.targets[val_data.indices])
    x_test = torch.tensor(dataset.attributes[test_data.indices])
    y_test = torch.tensor(dataset.targets[test_data.indices])

    # Network structures
    l1_weight = 1e-3
    hidden_neurons = [10, 5]

    # Setting device
    device = torch.device("cpu") if torch.cuda.is_available() else torch.device("cpu")
    set_seed(seed)

    print("Training General NN...")
    model = XGeneralNN(n_classes=dataset.n_classes, n_features=n_features, hidden_neurons=hidden_neurons,
                       loss=loss, l1_weight=l1_weight)

    results = model.fit(train_data, val_data, epochs=epochs, l_r=l_r, metric=metric,
                        lr_scheduler=lr_scheduler, device=device, save=False, verbose=False)
    print(results)
    accuracy = model.evaluate(test_data, metric=metric)
    print("Test model accuracy", accuracy)

    formulas, times, exp_accuracies = [], [], []
    for i, class_to_explain in enumerate(dataset.classes):
        formula, elapsed_time = model.get_global_explanation(x_val, y_val, i, simplify=True,
                                                             topk_explanations=top_k_explanations,
                                                             concept_names=concept_names, return_time=True)
        exp_accuracy, _ = test_multi_class_explanation(formula, i, x_test, y_test,
                                                       metric=metric, concept_names=concept_names)
        formulas.append(formula), times.append(elapsed_time), exp_accuracies.append(exp_accuracy)
        print(f"{class_to_explain} <-> {formula}")
        print("Elapsed time", elapsed_time)
        print("Explanation accuracy", exp_accuracy)

    methods.append("General")
    splits.append(seed)
    explanations.append(formulas[0])
    explanations_inv.append(formulas[1])
    model_accuracies.append(accuracy)
    explanation_accuracies.append(np.mean(exp_accuracies))
    explanation_accuracies_inv.append(np.mean(exp_accuracies))
    elapsed_times.append(np.mean(times))
    elapsed_times_inv.append(np.mean(times))

In [ ]:
results = pd.DataFrame({
    'method': methods,
    'split': splits,
    'explanation': explanations,
    'explanation_inv': explanations_inv,
    'model_accuracy': model_accuracies,
    'explanation_accuracy': explanation_accuracies,
    'explanation_accuracy_inv': explanation_accuracies_inv,
    'elapsed_time': elapsed_times,
    'elapsed_time_inv': elapsed_times_inv,
})
results_general = results[results['method'] == "General"]
results_general.to_csv(os.path.join(results_dir, 'results_general.csv'))
results.to_csv(os.path.join(results_dir, 'results.csv'))
print(results)

# Summary

In [4]:
cols = ['model_accuracy', 'explanation_accuracy', 'explanation_accuracy_inv', 'elapsed_time', 'elapsed_time_inv']
mean_cols = [f'{c}_mean' for c in cols]
sem_cols = [f'{c}_sem' for c in cols]

# general
results_general = pd.read_csv(os.path.join(results_dir, "results_general.csv"))
# results_general = results[results['method'] == "General"]
df_mean = results_general[cols].mean()
df_sem = results_general[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_pruning = pd.concat([df_mean, df_sem])
summary_pruning.name = 'general'

# # lime
# df_mean = results_lime[cols].mean()
# df_sem = results_lime[cols].sem()
# df_mean.columns = mean_cols
# df_sem.columns = sem_cols
# summary_lime = pd.concat([df_mean, df_sem])
# summary_lime.name = 'lime'

# relu
# results_relu = results[results['method'] == "Relu"]
results_relu = pd.read_csv(os.path.join(results_dir, "results_relu.csv"))
df_mean = results_relu[cols].mean()
df_sem = results_relu[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_weights = pd.concat([df_mean, df_sem])
summary_weights.name = 'relu'

# psi
# results_psi = results[results['method'] == "Psi"]
results_psi = pd.read_csv(os.path.join(results_dir, "results_psi.csv"))
df_mean = results_psi[cols].mean()
df_sem = results_psi[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_psi = pd.concat([df_mean, df_sem])
summary_psi.name = 'psi'

# tree
# results_tree = results[results['method'] == "Tree"]
results_tree = pd.read_csv(os.path.join(results_dir, "results_tree.csv"))
df_mean = results_tree[cols].mean()
df_sem = results_tree[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_tree = pd.concat([df_mean, df_sem])
summary_tree.name = 'tree'

summary = pd.concat([summary_pruning,
                     #                      summary_lime,
                     summary_weights,
                     summary_psi,
                     summary_tree], axis=1).T
summary.columns = mean_cols + sem_cols
print(summary)

         model_accuracy_mean  explanation_accuracy_mean  \
general            87.868504                  76.198678   
relu               92.138631                  92.835126   
psi                66.633704                  63.664694   
tree               89.036348                  89.036348   

         explanation_accuracy_inv_mean  elapsed_time_mean  \
general                      76.198678           0.071068   
relu                         92.835126           0.088810   
psi                          63.664694           0.039648   
tree                         89.036348           0.006912   

         elapsed_time_inv_mean  model_accuracy_sem  explanation_accuracy_sem  \
general               0.071068            0.270393                  1.584872   
relu                  0.088810            0.209345                  0.362614   
psi                   0.039648            1.273933                  0.499402   
tree                  0.006912            0.329574                  0.329574  

In [ ]:
summary.to_csv(os.path.join(results_dir, 'summary.csv'))

